# 04. Purged Walk-Forward Validation

## Цель
Реализовать Purged Walk-Forward Validation для избежания look-ahead bias при оценке классификаторов рыночных зон.

## Задачи
1. Реализовать Purged Walk-Forward Validation
2. Реализовать Time Series Cross-Validation
3. Реализовать Monte Carlo Validation
4. Протестировать на данных BTC
5. Сравнить результаты разных методов валидации


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
import os

# Добавляем путь к нашим модулям
sys.path.append('../evaluation')
sys.path.append('../classifiers')

from purged_walk_forward import PurgedWalkForward, TimeSeriesCrossValidation, MonteCarloValidation, ComprehensiveValidation
from economic_metrics import EconomicMetrics
from classifiers import MZAClassifier

print("Библиотеки загружены успешно!")
print(f"Текущая директория: {os.getcwd()}")


In [ ]:
# Загружаем данные BTC для тестирования
print("=== ЗАГРУЗКА ДАННЫХ BTC ===")

# Загружаем данные из папки indicators/data_frames
file_path = '../../indicators/data_frames/df_btc_1h.csv'
df_btc = pd.read_csv(file_path)
df_btc['timestamps'] = pd.to_datetime(df_btc['timestamps'])
df_btc.set_index('timestamps', inplace=True)

print(f"✅ Загружено {len(df_btc)} записей BTC")
print(f"Период: {df_btc.index[0]} - {df_btc.index[-1]}")

# Создаем объект для вычисления экономических метрик
economic_metrics = EconomicMetrics()

print("✅ Данные подготовлены для валидации")


In [ ]:
# Тестируем Purged Walk-Forward Validation
print("=== ТЕСТИРОВАНИЕ PURGED WALK-FORWARD VALIDATION ===")

# Создаем Purged Walk-Forward валидатор
purged_wf = PurgedWalkForward(n_splits=5, purge_days=2, embargo_days=1)

# Создаем MZA классификатор
mza = MZAClassifier()

# Оцениваем классификатор с Purged Walk-Forward
results_pwf = purged_wf.evaluate_classifier(
    mza, df_btc, economic_metrics.calculate_economic_metrics
)

print(f"✅ Purged Walk-Forward: {len(results_pwf)} разделений")

# Вычисляем метрики стабильности
stability_metrics = purged_wf.calculate_stability_metrics(results_pwf)

print("\n📊 Метрики стабильности:")
for key, value in stability_metrics.items():
    print(f"  {key}: {value:.4f}")

# Показываем результаты для каждого разделения
print("\n📈 Результаты по разделениям:")
for i, result in enumerate(results_pwf):
    print(f"  Разделение {i+1}:")
    print(f"    Bull return: {result['bull_return']:.4f}")
    print(f"    Bear return: {result['bear_return']:.4f}")
    print(f"    Return spread: {result['return_spread']:.4f}")
    print(f"    Economic value: {result['economic_value']:.4f}")


In [ ]:
# Тестируем Time Series Cross-Validation
print("=== ТЕСТИРОВАНИЕ TIME SERIES CROSS-VALIDATION ===")

# Создаем Time Series Cross-Validation валидатор
ts_cv = TimeSeriesCrossValidation(n_splits=5, test_size=0.2)

# Оцениваем классификатор с Time Series CV
results_ts = ts_cv.evaluate_classifier(
    mza, df_btc, economic_metrics.calculate_economic_metrics
)

print(f"✅ Time Series CV: {len(results_ts)} разделений")

# Показываем результаты для каждого разделения
print("\n📈 Результаты по разделениям:")
for i, result in enumerate(results_ts):
    print(f"  Разделение {i+1}:")
    print(f"    Bull return: {result['bull_return']:.4f}")
    print(f"    Bear return: {result['bear_return']:.4f}")
    print(f"    Return spread: {result['return_spread']:.4f}")
    print(f"    Economic value: {result['economic_value']:.4f}")


In [ ]:
# Тестируем Monte Carlo Validation
print("=== ТЕСТИРОВАНИЕ MONTE CARLO VALIDATION ===")

# Создаем Monte Carlo валидатор
monte_carlo = MonteCarloValidation(n_iterations=50, noise_level=0.01)

# Оцениваем классификатор с Monte Carlo
results_mc = monte_carlo.evaluate_classifier(
    mza, df_btc, economic_metrics.calculate_economic_metrics
)

print(f"✅ Monte Carlo: {len(results_mc)} итераций")

# Вычисляем метрики робастности
robustness_metrics = monte_carlo.calculate_robustness_metrics(results_mc)

print("\n📊 Метрики робастности:")
for key, value in robustness_metrics.items():
    print(f"  {key}: {value:.4f}")

# Показываем статистику результатов
print("\n📈 Статистика результатов:")
results_df = pd.DataFrame(results_mc)
for column in ['bull_return', 'bear_return', 'return_spread', 'economic_value']:
    if column in results_df.columns:
        values = results_df[column].dropna()
        print(f"  {column}:")
        print(f"    Среднее: {values.mean():.4f}")
        print(f"    Стандартное отклонение: {values.std():.4f}")
        print(f"    Минимум: {values.min():.4f}")
        print(f"    Максимум: {values.max():.4f}")


In [ ]:
# Комплексная валидация
print("=== КОМПЛЕКСНАЯ ВАЛИДАЦИЯ ===")

# Создаем комплексный валидатор
comprehensive = ComprehensiveValidation(
    n_splits=5, purge_days=2, embargo_days=1,
    n_iterations=50, noise_level=0.01
)

# Оцениваем классификатор с комплексной валидацией
results_comprehensive = comprehensive.evaluate_classifier(
    mza, df_btc, economic_metrics.calculate_economic_metrics
)

print(f"✅ Комплексная валидация завершена")

# Вычисляем комплексные метрики
comprehensive_metrics = comprehensive.calculate_comprehensive_metrics(results_comprehensive)

print("\n📊 Комплексные метрики:")
for key, value in comprehensive_metrics.items():
    print(f"  {key}: {value:.4f}")

# Показываем результаты всех методов
print("\n📈 Сравнение методов валидации:")
print(f"  Purged Walk-Forward: {len(results_comprehensive['purged_walk_forward'])} разделений")
print(f"  Time Series CV: {len(results_comprehensive['time_series_cv'])} разделений")
print(f"  Monte Carlo: {len(results_comprehensive['monte_carlo'])} итераций")
print(f"  Общее количество валидаций: {comprehensive_metrics['total_validations']}")
